In [1]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from utils import fraunhofer_distance, reflection_coefficient, estimate_apd

In [2]:
sns.set_theme(style='ticks',
              font_scale=1.5,
              palette='colorblind',
              rc={'text.usetex': True,
                  'text.latex.preamble': r'\usepackage{amsmath}',
                  'font.family': 'serif'})

In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# IPD

Why the IPD values differ from the data?

In [4]:
from dosipy.utils.integrate import elementwise_rectquad

In [17]:
d = 2
f = 90

In [18]:
# load computed ipd data
PDinc = np.load(os.path.join('data',
                             f'02_power_density_d{d}mm_f{f}GHz.npy'))
PDinc_n = PDinc[:, :, 2].real
PDinc_tot = np.sqrt(PDinc[:, :, 0].real ** 2
                    + PDinc[:, :, 1].real ** 2
                    + PDinc[:, :, 2].real ** 2)
lims = (int(PDinc.shape[0]/4),
        int(PDinc.shape[0]*3/4) + 1)

# extent of the exposed surface in mm
extent = (-10, 10, -10, 10)

# averaged incident power densities
A4 = 4 / 100 / 100  # 4 cm2 integration surface
A1 = A4 / 4  # 1 cm2 integration surface
x = np.linspace(extent[0], extent[1], PDinc.shape[0]) / 1000
y = np.linspace(extent[2], extent[3], PDinc.shape[1]) / 1000
sPDinc_n1 = elementwise_rectquad(x[lims[0]:lims[1]],
                                 y[lims[0]:lims[1]],
                                 PDinc_n[lims[0]:lims[1],
                                         lims[0]:lims[1]]) / (2 * A1)
sPDinc_tot1 = elementwise_rectquad(x[lims[0]:lims[1]],
                                   y[lims[0]:lims[1]],
                                   PDinc_tot[lims[0]:lims[1],
                                             lims[0]:lims[1]]) / (2 * A1)
sPDinc_n4 = elementwise_rectquad(x, y, PDinc_n) / (2 * A4)
sPDinc_tot4 = elementwise_rectquad(x, y, PDinc_tot) / (2 * A4)

In [19]:
print(d, f, 'normal', '\t', 1, '\t',  sPDinc_n1)
print(d, f, 'norm', '\t', 1, '\t', sPDinc_tot1)
print(d, f, 'normal', '\t', 4, '\t', sPDinc_n4)
print(d, f, 'norm', '\t', 4, '\t', sPDinc_tot4)

2 90 normal 	 1 	 44.89596811880829
2 90 norm 	 1 	 63.19145355783379
2 90 normal 	 4 	 13.81421499847122
2 90 norm 	 4 	 23.13932013768433


In [20]:
# load the ipd data from Li et al. 2022
ipd = pd.read_csv(os.path.join('data', '03_Li_2022_data_table_6.csv'))
print(ipd[(ipd['d'] == d) & (ipd['f'] == f)])

    d   f definition  area   mean   std
24  2  90     normal     1  36.99  0.90
25  2  90       norm     1  56.53  1.67
26  2  90     normal     4  10.83  0.39
27  2  90       norm     4  20.38  0.99


# APD

Update `estimate_apd` method:
1. should the APD distribution be scaled prior to the spatial averaging?
2. if the answer to 1. is $yes$, how the angle of incidence should be considered?

N.B. For the comparison of the estimated values with the data from Li $et al$. 2023, consider only the 1-layer skin model at 10 GHz; at 30 and 90 GHz, both the 1- and 3-layer skin model should be considered. There are large discrepancies between the 1- and 3-layer model at 10 GHz because deeper tissue layers come into play because of the penetration depth which is considerably greater than at 30 and 90 GHz. At higher frequencies, there are only marginal differences between corresponding APD values on 1- and 3-layer skin model.

In [ ]:
# load the apd data
apd = pd.read_csv(os.path.join('data', '03_Li_2022_data_table_8.csv'))

# skin conductivity in S/m2 at 10, 30, and 90 GHz
sigma = {10: 8.48,
         30: 27.31,
         90: 41.94}

# relative dielectric permittivity of skin at 10, 30, and 90 GHz
eps_r = {10: 32.41,
         30: 16.63,
         90: 6.83}

# extent of the exposed surface in mm
extent = (-10, 10, -10, 10)

data_n = []
data_tot = []
for d, f in apd.groupby(['d', 'f']).size().reset_index()[['d', 'f']].values:
    sPDab_n1, sPDab_tot1, sPDab_n4, sPDab_tot4 = estimate_apd(d, f, extent,
                                                              sigma=sigma[f],
                                                              eps_r=eps_r[f])
    data_n.append([d, f, None, 1, sPDab_n1, None])
    data_n.append([d, f, None, 4, sPDab_n4, None])
    data_tot.append([d, f, None, 1, sPDab_tot1, None])
    data_tot.append([d, f, None, 4, sPDab_tot4, None])

apd_n = pd.DataFrame(data=data_n, columns=apd.columns)
apd_tot = pd.DataFrame(data=data_tot, columns=apd.columns)

In [ ]:
f = 10
layers = 1

cs = sns.color_palette('rocket', 2)
fig, ax = plt.subplots()

# collected data
data = apd[(apd['f']==f) & (apd['layers']==layers)]
d = data[data['area'] == 4]['d']
mean_4 = data[data['area'] == 4]['mean']
std_4 = data[data['area'] == 4]['std']
mean_1 = data[data['area'] == 1]['mean']
std_1 = data[data['area'] == 1]['std']
ax.errorbar(d, mean_4, 2*std_4,
            ls='-', lw=3, c=cs[0],
            marker='o', ms=7, mfc='white', mew=1, mec=cs[0],
            capsize=3)
ax.errorbar(d, mean_1, 2*std_1,
            ls='-', lw=3, c=cs[0],
            marker='s', ms=7, mfc='white', mew=1, mec=cs[0],
            capsize=3)

# estimated data
data_n = apd_n[(apd_n['f']==f)]
estim_4 = data_n[data_n['area'] == 4]['mean']
estim_1 = data_n[data_n['area'] == 1]['mean']
ax.plot(d, estim_4,
        ls='--', lw=3, c=cs[1],
        marker='o', ms=7, mfc='white', mew=1,
        zorder=3)
ax.plot(d, estim_1,
        ls='--', lw=3, c=cs[1],
        marker='s', ms=7, mfc='white', mew=1,
        zorder=4)

# figure settings
ymax = (mean_1+2*std_1).max()
ytick = np.ceil(ymax) if np.ceil(ymax) % 2 == 0 else np.floor(ymax)
ax.set(xlabel='distance (mm)',
       xticks=d,
       xticklabels=d,
       ylabel='power density (W/m$^2$)',
       yticks=[0, ytick/2, ytick],
       yticklabels=[0, int(ytick/2), int(ytick)],
      )
ax.plot([], [], 'ko', ms=7, mfc='white', mew=1, label='$A$ = 4 cm$^2$')
ax.plot([], [], 'ks', ms=7, mfc='white', mew=1, label='$A$ = 1 cm$^2$')
ax.plot([], [], '-', lw=3, c=cs[0], label='$sAPD$')
ax.plot([], [], '--', lw=3, c=cs[1], label=r'$\widetilde{sAPD}$')
ax.legend(loc='best', frameon=False)
sns.despine(fig=fig, ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
# tba